In [ ]:
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import os
import math
import subprocess
import time

from scipy.optimize import curve_fit
from scipy.special import gamma
from scipy.special import gammaincc

In [ ]:
# File pathways necessary for running
GARFIELDSOURCE  = '/Users/tannerp/Work/garfield/install/share/Garfield/setupGarfield.sh'

In [ ]:
def setupSimulation():
    """
    """
    if not os.path.exists("build"):
        os.makedirs("build")

    makeBuild = (
        f'source {GARFIELDSOURCE} && '
        f'cd build && '
        f'cmake .. && '
        f'make'
    )
    result = subprocess.run(
        makeBuild,
        shell=True,
        check=True,
        executable='/bin/bash',
        capture_output=True,
        text=True
    )
    return
        
setupSimulation()    

In [ ]:
def defaultParam():
    """
    """
    defaultParam = {
        'pixelWidth': 10.,
        'pitch': 55.,
        'meshStandoff': 100.,
        'meshThickness': 5.,
        'holeRadius': 16.,
        'cathodeHeight': 400.,
        'thicknessSiO2': 5.,
        'fieldRatio': 25.,
        'numFieldLine': 25,
        'transparencyLimit': .1,
        'numAvalanche': 1000,
        'avalancheLimit': 200,
        'gasCompAr': 80.,
        'gasCompCO2': 20.,
    }
    return defaultParam

In [ ]:
def readParam():
    """
    """
    filename = 'runControl'
    param = {}
    
    try:
        with open(filename, 'r') as file:
            for line in file:
                line = line.strip()  # Remove leading/trailing whitespace
                if line.startswith('/') or not line:  # Skip comments and empty lines
                    continue
                # Split the line at the '='
                parts = line.split('=', 1)
                if len(parts) == 2:
                    key, value = parts[0].strip(), parts[1].strip()
                    value = value.rstrip(';') # Remove trailing semicolon
                    param[key] = value
                else:
                    print(f"Skipping malformed line: {line}")
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return {}
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return {}

    return param

In [ ]:
def writeRunControl(param=None):
    """
    """
    filename = 'runControl'

    if param is None or not param:
        print(f"No parameters to write to {filename}.")
        return

    try:
        with open(filename, 'r') as file:
            oldLines = file.readlines()  # Read all lines of the file
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return

    newLines = []
    for line in oldLines:
        line = line.strip()
        if line.startswith('/') or not line:
            newLines.append(line + '\n')  # Keep comments and empty lines
            continue

        parts = line.split('=', 1)
        if len(parts) == 2:
            key = parts[0].strip()
            if key in param:
                newLines.append(f"{key} = {param[key]};\n")  # Update value
            else:
                newLines.append(line + '\n') #keep original line
        else:
            newLines.append(line + '\n')  # Keep non-parameter lines

    try:
        with open(filename, 'w') as file:
            file.writelines(newLines)
    except Exception as e:
        print(f"An error occurred while writing to {filename}: {e}")
    
    return

In [ ]:
def calcPotentials(param=None):
    """
    """

    if param is None or not param:
        print(f"Invalid parameters.")
        return None
        
    convertEField = 0.1 # 1 kV/cm = 0.1 V/micron
    
    # Calculate the voltage required to achieve amplification field
    meshDistance = float(param['meshStandoff']) - float(param['meshThickness'])/2. #micron

    meshVoltage = float(param['fieldRatio'])*convertEField*meshDistance

    # Calculate for drift field
    cathodeDistance = float(param['cathodeHeight']) - float(param['meshThickness'])/2. #micron
    cathodeVoltage = convertEField*cathodeDistance + meshVoltage

    potentials = {
        'cathodeVoltage': -cathodeVoltage,
        'meshVoltage': -meshVoltage
    }
    
    return potentials

In [ ]:
def writeSIF(param=None):
    """
    """
    filename = os.path.join('./Geometry', 'FIMS.sif')

    if param is None or not param:
        print(f"Invalid parameters.")
        return None
    
    try:
        with open(filename, 'r') as file:
            sifLines = file.readlines()  # Read all lines of the file

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return

    potentials = calcPotentials(param)

    writeCathode = -1
    writeMesh = -1

    # Find the cathod and mesh naming lines
    for i, line in enumerate(sifLines):
        if 'Name = "Cathode"' in line:
            writeCathode = i+1
        if 'Name = "Mesh"' in line:
            writeMesh = i+1

    if writeCathode == -1 or 'Potential =' not in sifLines[writeCathode]:
        print('Error with cathode.')
        return
    if writeMesh == -1 or 'Potential =' not in sifLines[writeMesh]:
        print('Error with mesh.')
        return
    
    sifLines[writeCathode] = f'\tPotential = {potentials['cathodeVoltage']}\n'
    sifLines[writeMesh] = f'\tPotential = {potentials['meshVoltage']}\n'
        
    try:
        with open(filename, 'w') as file:
            file.writelines(sifLines)
    except Exception as e:
        print(f"An error occurred while writing to {filename}: {e}")
        return
        
    return

In [ ]:
def writeParam(param=None):
    """
    """
    if param is None or not param:
        print(f"No parameters to write.")
        return    

    writeRunControl(param)
    writeSIF(param)
    
    return

In [ ]:
def resetParam():
    """
    """
    defaultParams = defaultParam()
    writeParam(defaultParams)

    print('Parameters have been reset.')
    
    return defaultParams

In [ ]:
def runGmsh():
    with open(os.path.join(os.getcwd(), 'log/logGmsh.txt'), 'w+') as gmshOutput:
        startTime = time.monotonic()
        subprocess.run(['gmsh', os.path.join('./Geometry/', 'FIMS.txt'), '-3'], stdout=gmshOutput)
        endTime = time.monotonic()
        gmshOutput.write(f'\n\nGmsh run time: {endTime - startTime} s')
    return

def runElmer():
    originalCWD = os.getcwd()
    os.chdir('./Geometry')
    try:
        with open(os.path.join(originalCWD, 'log/logElmerGrid.txt'), 'w+') as elmerOutput:
            startTime = time.monotonic()
            subprocess.run(['ElmerGrid', '14', '2', 'FIMS.msh', '-out', '.', '-autoclean'], stdout=elmerOutput)
            endTime = time.monotonic()
            elmerOutput.write(f'\n\nElmerGrid run time: {endTime - startTime} s')
            
        with open(os.path.join(originalCWD, 'log/logElmerSolver.txt'), 'w+') as elmerOutput:
            startTime = time.monotonic()
            subprocess.run(['ElmerSolver', 'FIMS.sif'], stdout=elmerOutput)
            endTime = time.monotonic()
            elmerOutput.write(f'\n\nElmerSolver run time: {endTime - startTime} s')
    finally:
        os.chdir(originalCWD)
    return

def runGarfield():
    originalCWD = os.getcwd()
    os.chdir('./build/')
    try:
        with open(os.path.join(originalCWD, 'log/logGarfield.txt'), 'w+') as garfieldOutput:
            startTime = time.monotonic()
            setupAvalanche = (
                f'source {GARFIELDSOURCE} && '
                f'make && '
                f'./runAvalanche'
            )
            subprocess.run(setupAvalanche, stdout=garfieldOutput, shell=True, check=True)
            endTime = time.monotonic()
            garfieldOutput.write(f'\n\nGarfield run time: {endTime - startTime} s')
    finally:
        os.chdir(originalCWD)
    return

def runSimulation(param=None):

    if param is None or not param:
        print(f"No parameters to run.")
        return   
        
    writeParam(param)
    runGmsh()
    runElmer()
    runGarfield()
    return

In [ ]:
def getRunNumber():
    """
    """
    filename = 'runNo'

    try:
        with open(filename, 'r') as file:
            content = file.read().strip()
            runNo = int(content)
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return
    except ValueError:
        print(f"Error: Invalid number format in '{filename}")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return
    
    return runNo

In [ ]:
def varyParam(parameter, minVal, maxVal, numSteps=10):
    """
    """
    
    resetParam()
    param = readParam()
    
    if parameter not in param:
        print(f"Parameter '{parameter}' not able to be varied.")
        return None, None

    runNo = getRunNumber()
    print(f"Varying '{parameter}'. Initial run number: {runNo}")

    variedParam = np.linspace(minVal, maxVal, numSteps)
    

    for inParam in variedParam:
        param[parameter] = inParam

        runNo = getRunNumber()
        runSimulation(param)
    
        print(f'\tDone run {runNo}.')

    runNo = getRunNumber()
    print(f"Done '{parameter}'. Final run number: {runNo-1}")

    param = resetParam()

    return variedParam

In [ ]:
varyParam('holeRadius', 14, 15, 1)